In [7]:
# Mount Google Drive (optional, if dataset is stored there)
from google.colab import drive
drive.mount('/content/drive')


!unzip /content/data.zip -d /content/dataset


Mounted at /content/drive
Archive:  /content/data.zip
   creating: /content/dataset/data/
  inflating: /content/dataset/data/.gitkeep  
   creating: /content/dataset/data/MNIST/
  inflating: /content/dataset/data/MNIST/dataset.hdf5  
   creating: /content/dataset/data/MNIST/testing/
  inflating: /content/dataset/data/MNIST/testing/dataORG_0.mat  
  inflating: /content/dataset/data/MNIST/testing/dataORG_1.mat  
  inflating: /content/dataset/data/MNIST/testing/dataORG_2.mat  
  inflating: /content/dataset/data/MNIST/testing/dataORG_3.mat  
  inflating: /content/dataset/data/MNIST/testing/dataORG_4.mat  
  inflating: /content/dataset/data/MNIST/testing/dataORG_5.mat  
  inflating: /content/dataset/data/MNIST/testing/dataORG_6.mat  
  inflating: /content/dataset/data/MNIST/testing/dataORG_7.mat  
  inflating: /content/dataset/data/MNIST/testing/dataORG_8.mat  
  inflating: /content/dataset/data/MNIST/testing/dataORG_9.mat  
   creating: /content/dataset/data/MNIST/training/
  inflating: /c

In [ ]:
#shallow CNN
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
import numpy as np
import os
import h5py
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
from PIL import Image

# Dataset loading and processing functions
def load_data(dataset, num_classes, datadir='../data'):
    cache_file = os.path.join(datadir, dataset, 'dataset.hdf5')
    if os.path.exists(cache_file):
        with h5py.File(cache_file, 'r') as f:
            x_train, y_train = f['x_train'][()], f['y_train'][()]
            x_test, y_test = f['x_test'][()], f['y_test'][()]
            print('loaded from cache file data: x_train {} x_test {}'.format(x_train.shape, x_test.shape))
            return (x_train, y_train), (x_test, y_test)

    print('loading data from mat files')
    x_train, y_train, x_test, y_test = [], [], [], []
    for split in ['training', 'testing']:
        for classidx in range(num_classes):
            datafile = os.path.join(datadir, dataset, '{}/dataORG_{}.mat'.format(split, classidx))
            data = loadmat(datafile)['xxO'].transpose([2, 0, 1])  # (N, H, W)
            label = np.zeros(data.shape[0], dtype=np.int64) + classidx
            print('split {} class {} data.shape {}'.format(split, classidx, data.shape))
            if split == 'training':
                x_train.append(data)
                y_train.append(label)
            else:
                x_test.append(data)
                y_test.append(label)

    x_train, y_train = np.concatenate(x_train), np.concatenate(y_train)
    x_test, y_test = np.concatenate(x_test), np.concatenate(y_test)
    print('x_train.shape {} x_test.shape {}'.format(x_train.shape, x_test.shape))

    x_train = x_train / x_train.max(axis=(1, 2), keepdims=True)
    x_test = x_test / x_test.max(axis=(1, 2), keepdims=True)

    x_train = (x_train * 255.).astype(np.uint8)
    x_test = (x_test * 255.).astype(np.uint8)

    with h5py.File(cache_file, 'w') as f:
        f.create_dataset('x_train', data=x_train)
        f.create_dataset('y_train', data=y_train)
        f.create_dataset('x_test', data=x_test)
        f.create_dataset('y_test', data=y_test)
        print('saved to {}'.format(cache_file))

    return (x_train, y_train), (x_test, y_test)

# Preprocessing for CNN
def preprocess_data(x_train, x_test, img_size, use_subset=False):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),  # keeps it grayscale (1 channel)
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    x_train_transformed = [transform(x) for x in x_train]
    x_test_transformed = [transform(x) for x in x_test]

    x_train_tensor = torch.stack(x_train_transformed)
    x_test_tensor = torch.stack(x_test_transformed)

    y_train_tensor = torch.tensor(y_train)
    y_test_tensor = torch.tensor(y_test)

    if use_subset:
        subset_size = int(0.1 * len(x_train_tensor))
        x_train_tensor = x_train_tensor[:subset_size]
        y_train_tensor = y_train_tensor[:subset_size]

    return x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor

# 🧠 Shallow CNN Model Definition
class ShallowCNN(nn.Module):
    def __init__(self, num_classes):
        super(ShallowCNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # Output: 16x28x28
            nn.ReLU(),
            nn.MaxPool2d(2),  # Output: 16x14x14

            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # Output: 32x14x14
            nn.ReLU(),
            nn.MaxPool2d(2)  # Output: 32x7x7
        )
        self.fc_block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block(x)
        return x

# Training function
def train(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss:.4f} - Accuracy: {100*correct_preds/total_preds:.2f}%")

# Evaluation function
def evaluate(model, test_loader, device):
    model.eval()
    correct_preds = 0
    total_preds = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

    print(f'Test Accuracy: {100 * correct_preds / total_preds:.2f}%')

# Save model
def save_model(model, filename='shallow_cnn.pth'):
    torch.save(model.state_dict(), filename)
    print(f'Model saved as {filename}')

# Main Execution
if __name__ == '__main__':
    datadir = '/content/dataset/data'
    dataset = 'MNIST'
    num_classes = 10
    img_size = 28
    use_subset = True  # Use 10% of training data

    (x_train, y_train), (x_test, y_test) = load_data(dataset, num_classes, datadir)
    x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor = preprocess_data(x_train, x_test, img_size, use_subset=use_subset)

    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ShallowCNN(num_classes=num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    train(model, train_loader, criterion, optimizer, device, num_epochs=10)
    evaluate(model, test_loader, device)
    save_model(model)

    # Optional download in Colab
    from google.colab import files
    files.download('shallow_cnn.pth')


loaded from cache file data: x_train (60000, 28, 28) x_test (10000, 28, 28)
Epoch 1/10 - Loss: 11.9409 - Accuracy: 98.82%
Epoch 2/10 - Loss: 0.4212 - Accuracy: 99.90%
Epoch 3/10 - Loss: 0.3178 - Accuracy: 99.93%
Epoch 4/10 - Loss: 0.1313 - Accuracy: 99.98%
Epoch 5/10 - Loss: 0.1540 - Accuracy: 99.97%
Epoch 6/10 - Loss: 0.1931 - Accuracy: 99.95%
Epoch 7/10 - Loss: 0.1822 - Accuracy: 99.97%
Epoch 8/10 - Loss: 0.0466 - Accuracy: 100.00%
Epoch 9/10 - Loss: 0.0105 - Accuracy: 100.00%
Epoch 10/10 - Loss: 0.0094 - Accuracy: 100.00%
Test Accuracy: 21.09%
Model saved as shallow_cnn.pth


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#densenet121
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms, models
import numpy as np
import os
import h5py
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
from PIL import Image

# Dataset loading and processing functions
def load_data(dataset, num_classes, datadir='../data'):
    cache_file = os.path.join(datadir, dataset, 'dataset.hdf5')
    if os.path.exists(cache_file):
        with h5py.File(cache_file, 'r') as f:
            x_train, y_train = f['x_train'][()], f['y_train'][()]
            x_test, y_test = f['x_test'][()], f['y_test'][()]
            print('loaded from cache file data: x_train {} x_test {}'.format(x_train.shape, x_test.shape))
            return (x_train, y_train), (x_test, y_test)

    print('loading data from mat files')
    x_train, y_train, x_test, y_test = [], [], [], []
    for split in ['training', 'testing']:
        for classidx in range(num_classes):
            datafile = os.path.join(datadir, dataset, '{}/dataORG_{}.mat'.format(split, classidx))
            data = loadmat(datafile)['xxO'].transpose([2, 0, 1])
            label = np.zeros(data.shape[0], dtype=np.int64) + classidx
            print('split {} class {} data.shape {}'.format(split, classidx, data.shape))
            if split == 'training':
                x_train.append(data)
                y_train.append(label)
            else:
                x_test.append(data)
                y_test.append(label)

    x_train, y_train = np.concatenate(x_train), np.concatenate(y_train)
    x_test, y_test = np.concatenate(x_test), np.concatenate(y_test)

    x_train = x_train / x_train.max(axis=(1, 2), keepdims=True)
    x_test = x_test / x_test.max(axis=(1, 2), keepdims=True)

    x_train = (x_train * 255.).astype(np.uint8)
    x_test = (x_test * 255.).astype(np.uint8)

    with h5py.File(cache_file, 'w') as f:
        f.create_dataset('x_train', data=x_train)
        f.create_dataset('y_train', data=y_train)
        f.create_dataset('x_test', data=x_test)
        f.create_dataset('y_test', data=y_test)
        print('saved to {}'.format(cache_file))

    return (x_train, y_train), (x_test, y_test)

# Preprocessing for DenseNet121
def preprocess_data(x_train, x_test, img_size, use_subset=False):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((img_size, img_size)),
        transforms.Grayscale(num_output_channels=3),  # Convert 1-channel to 3-channel
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])

    x_train_transformed = [transform(x) for x in x_train]
    x_test_transformed = [transform(x) for x in x_test]

    x_train_tensor = torch.stack(x_train_transformed)
    x_test_tensor = torch.stack(x_test_transformed)

    y_train_tensor = torch.tensor(y_train)
    y_test_tensor = torch.tensor(y_test)

    if use_subset:
        subset_size = int(0.1 * len(x_train_tensor))
        x_train_tensor = x_train_tensor[:subset_size]
        y_train_tensor = y_train_tensor[:subset_size]

    return x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor

# Training function
def train(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss:.4f} - Accuracy: {100*correct_preds/total_preds:.2f}%")

# Evaluation function
def evaluate(model, test_loader, device):
    model.eval()
    correct_preds = 0
    total_preds = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

    print(f'Test Accuracy: {100 * correct_preds / total_preds:.2f}%')

# Save model
def save_model(model, filename='densenet121.pth'):
    torch.save(model.state_dict(), filename)
    print(f'Model saved as {filename}')

# Main Execution
if __name__ == '__main__':
    datadir = '/content/dataset/data'
    dataset = 'MNIST'
    num_classes = 10
    img_size = 32  # Adjusted for DenseNet121
    use_subset = True

    (x_train, y_train), (x_test, y_test) = load_data(dataset, num_classes, datadir)
    x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor = preprocess_data(x_train, x_test, img_size, use_subset=use_subset)

    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load DenseNet121 and replace the classifier
    model = models.densenet121(pretrained=False)
    model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    train(model, train_loader, criterion, optimizer, device, num_epochs=10)
    evaluate(model, test_loader, device)
    save_model(model, filename='densenet121.pth')

    # Optional: download model file in Colab
    from google.colab import files
    files.download('densenet121.pth')


loaded from cache file data: x_train (60000, 28, 28) x_test (10000, 28, 28)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10 - Loss: 9.4890 - Accuracy: 99.00%
Epoch 2/10 - Loss: 2.2732 - Accuracy: 99.63%
Epoch 3/10 - Loss: 1.1965 - Accuracy: 99.87%
Epoch 4/10 - Loss: 0.4749 - Accuracy: 99.95%
Epoch 5/10 - Loss: 0.6852 - Accuracy: 99.93%
Epoch 6/10 - Loss: 0.2423 - Accuracy: 99.97%
Epoch 7/10 - Loss: 0.8900 - Accuracy: 99.87%
Epoch 8/10 - Loss: 0.3021 - Accuracy: 99.95%
Epoch 9/10 - Loss: 0.9862 - Accuracy: 99.88%
Epoch 10/10 - Loss: 0.8869 - Accuracy: 99.92%
Test Accuracy: 21.10%
Model saved as densenet121.pth


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Rrcdt_nsc
import numpy as np
import numpy.linalg as LA
import pickle
from sklearn.metrics import accuracy_score

from scipy.io import loadmat
import os
import h5py
from PIL import Image
from sklearn.model_selection import train_test_split

!pip install pytranskit

from pytranskit.classification.rcdt_ns import RCDT_NS
from pytranskit.optrans.continuous.radoncdt import RadonCDT


def new_index_matrix(max_index, n_samples_perclass, num_classes, repeat, y_train):
    seed = int(f'{n_samples_perclass}{num_classes}{repeat}')
    np.random.seed(seed)
    index = np.zeros([num_classes, n_samples_perclass], dtype=np.int64)
    for classidx in range(num_classes):
        max_samples = (y_train == classidx).sum()
        index[classidx] = np.random.randint(0, max_samples, (n_samples_perclass))
    return index

def resize(X, target_size):
    assert len(X.shape) == 3
    if target_size == X.shape[1]:
        return X
    X_resize = []
    for i in range(X.shape[0]):
        im = Image.fromarray(X[i])
        im_resize = im.resize((target_size, target_size))
        X_resize.append(np.asarray(im_resize))
    X_resize = np.stack(X_resize, axis=0)
    assert X_resize.shape[0] == X.shape[0]
    return X_resize

def take_samples(data, labels, index, num_classes):
    assert data.shape[0] == labels.shape[0]
    assert index.shape[0] == num_classes
    indexed_data = []
    new_labels = []
    for i in range(num_classes):
        class_data, class_labels = data[labels == i], labels[labels == i]
        indexed_data.append(class_data[index[i]])
        new_labels.append(class_labels[index[i]])
    return np.concatenate(indexed_data), np.concatenate(new_labels)

def load_data(dataset, num_classes, datadir='../data'):
    cache_file = os.path.join(datadir, dataset, 'dataset.hdf5')
    if os.path.exists(cache_file):
        with h5py.File(cache_file, 'r') as f:
            x_train, y_train = f['x_train'][()], f['y_train'][()]
            x_test, y_test = f['x_test'][()], f['y_test'][()]
            print('loaded from cache file data: x_train {} x_test {}'.format(x_train.shape, x_test.shape))
            return (x_train, y_train), (x_test, y_test)

    print('loading data from mat files')
    x_train, y_train, x_test, y_test = [], [], [], []
    for split in ['training', 'testing']:
        for classidx in range(num_classes):
            datafile = os.path.join(datadir, dataset, f'{split}/dataORG_{classidx}.mat')
            data = loadmat(datafile)['xxO'].transpose([2, 0, 1])
            label = np.zeros(data.shape[0], dtype=np.int64) + classidx
            print(f'split {split} class {classidx} data.shape {data.shape}')
            if split == 'training':
                x_train.append(data)
                y_train.append(label)
            else:
                x_test.append(data)
                y_test.append(label)
    x_train, y_train = np.concatenate(x_train), np.concatenate(y_train)
    x_test, y_test = np.concatenate(x_test), np.concatenate(y_test)
    print(f'x_train.shape {x_train.shape} x_test.shape {x_test.shape}')

    x_train = x_train / x_train.max(axis=(1, 2), keepdims=True)
    x_test = x_test / x_test.max(axis=(1, 2), keepdims=True)

    x_train = (x_train * 255.).astype(np.uint8)
    x_test = (x_test * 255.).astype(np.uint8)

    with h5py.File(cache_file, 'w') as f:
        f.create_dataset('x_train', data=x_train)
        f.create_dataset('y_train', data=y_train)
        f.create_dataset('x_test', data=x_test)
        f.create_dataset('y_test', data=y_test)
        print(f'saved to {cache_file}')

    return (x_train, y_train), (x_test, y_test)

def load_data_3D(dataset, num_classes):
    (x_train, y_train), (x_test, y_test) = load_data(dataset, num_classes)
    x_train = x_train.reshape(-1, 1, x_train.shape[1], x_train.shape[2])
    x_train = np.repeat(x_train, axis=1, repeats=3)

    x_test = x_test.reshape(-1, 1, x_test.shape[1], x_test.shape[2])
    x_test = np.repeat(x_test, axis=1, repeats=3)
    return (x_train, y_train), (x_test, y_test)

def take_train_samples(x_train, y_train, n_samples_perclass, num_classes, repeat):
    max_index = x_train.shape[0] // num_classes
    train_index = new_index_matrix(max_index, n_samples_perclass, num_classes, repeat, y_train)
    x_train_sub, y_train_sub = take_samples(x_train, y_train, train_index, num_classes)
    return x_train_sub, y_train_sub

def dataset_config(dataset):
    assert dataset in ['OASIS1_age', 'OASIS1_dx', 'HEP2', 'AffMNIST', 'LiverN', 'MNIST', 'OAM', 'OAM_t5', 'OAM_t10', 'SignMNIST', 'Synthetic', 'Synthetic100', 'CIFAR10', 'MNIST_outDist']
    if dataset in ['MNIST']:
        rm_edge = True
        num_classes = 10
        po_train_max = 12
        img_size = 28
    elif dataset in ['AffMNIST']:
        rm_edge = True
        num_classes = 10
        img_size = 84
        po_train_max = 12
    elif dataset in ['OAM', 'OAM_t10', 'OAM_t5']:
        rm_edge = False
        num_classes = 32
        img_size = 151
        po_train_max = 9
    elif dataset in ['SignMNIST']:
        rm_edge = False
        num_classes = 3
        img_size = 128
        po_train_max = 9
    elif dataset in ['Synthetic']:
        rm_edge = True
        num_classes = 1000
        img_size = 64
        po_train_max = 4
    elif dataset in ['LiverN']:
        rm_edge = False
        num_classes = 2
        img_size = 130
        po_train_max = 8
    elif dataset in ['CIFAR10']:
        rm_edge = False
        num_classes = 10
        img_size = 32
        po_train_max = 12
    elif dataset in ['MNIST_outDist']:
        rm_edge = True
        num_classes = 10
        img_size = 84
        po_train_max = 12
    elif dataset in ['HEP2']:
        rm_edge = False
        num_classes = 2
        img_size = 64
        po_train_max = 10
    elif dataset in ['OASIS1_age']:
        rm_edge = False
        num_classes = 2
        img_size = 208
        po_train_max = 6
    elif dataset in ['OASIS1_dx']:
        rm_edge = False
        num_classes = 2
        img_size = 208
        po_train_max = 5
    elif dataset in ['Synthetic100']:
        rm_edge = True
        num_classes = 100
        img_size = 64
        po_train_max = 6

    return num_classes, img_size, po_train_max, rm_edge


# -------------------- Main Execution --------------------

if __name__ == '__main__':
    datadir = '/content/dataset/data'  # Adjust path if needed
    dataset = 'MNIST'
    num_classes, img_size, po_train_max, rm_edge = dataset_config(dataset)

    theta = np.linspace(0, 176, 45)
    (x_train, y_train), (x_test, y_test) = load_data(dataset, num_classes, datadir)

    model_filename = 'Rrcdt_ns_model(bangtestset2).pkl'
    percentage = 0.1
    n_samples_perclass = int(len(x_train) * percentage)
    num_repeats = 1

    accs = []
    train_accs = []

    for repeat in range(num_repeats):
        x_train_sub, y_train_sub = take_train_samples(x_train, y_train, n_samples_perclass, num_classes, repeat)

        classifier = RCDT_NS(num_classes, theta, rm_edge)
        classifier.fit(x_train_sub, y_train_sub)

        train_preds = classifier.predict(x_train_sub)
        test_preds = classifier.predict(x_test)

        train_acc = accuracy_score(y_train_sub, train_preds)
        test_acc = accuracy_score(y_test, test_preds)

        train_accs.append(train_acc)
        accs.append(test_acc)

        print(f'\nRepeat {repeat+1}/{num_repeats}')
        print(f'Training Accuracy: {train_acc:.4f}')
        print(f'Testing Accuracy:  {test_acc:.4f}')

    print(f'\nMean Training Accuracy over {num_repeats} run(s): {np.mean(train_accs):.4f}')
    print(f'Mean Testing Accuracy over {num_repeats} run(s):  {np.mean(accs):.4f}')

    # Save model
    with open(model_filename, 'wb') as f:
        pickle.dump(classifier, f)
    print(f'Model saved as {model_filename}')

    # Download model
    from google.colab import files
    files.download(model_filename)


loaded from cache file data: x_train (60000, 28, 28) x_test (10000, 28, 28)

Calculating RCDTs for training images ...
Generating basis vectors for each class ...

Calculating RCDTs for testing images ...
Finding nearest subspace for each test sample ...

Calculating RCDTs for testing images ...
Finding nearest subspace for each test sample ...

Repeat 1/1
Training Accuracy: 0.9984
Testing Accuracy:  0.9534

Mean Training Accuracy over 1 run(s): 0.9984
Mean Testing Accuracy over 1 run(s):  0.9534
Model saved as Rrcdt_ns_model(bangtestset2).pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
#resnet18with time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms
import numpy as np
import os
import h5py
from scipy.io import loadmat
import time
import psutil
from PIL import Image


# Dataset loading and processing functions
def load_data(dataset, num_classes, datadir='../data'):
    cache_file = os.path.join(datadir, dataset, 'dataset.hdf5')
    if os.path.exists(cache_file):
        with h5py.File(cache_file, 'r') as f:
            x_train, y_train = f['x_train'][()], f['y_train'][()]
            x_test, y_test = f['x_test'][()], f['y_test'][()]
            print('loaded from cache file data: x_train {} x_test {}'.format(x_train.shape, x_test.shape))
            return (x_train, y_train), (x_test, y_test)

    print('loading data from mat files')
    x_train, y_train, x_test, y_test = [], [], [], []
    for split in ['training', 'testing']:
        for classidx in range(num_classes):
            datafile = os.path.join(datadir, dataset, '{}/dataORG_{}.mat'.format(split, classidx))
            data = loadmat(datafile)['xxO'].transpose([2, 0, 1])  # transpose to (N, H, W)
            label = np.zeros(data.shape[0], dtype=np.int64) + classidx
            print('split {} class {} data.shape {}'.format(split, classidx, data.shape))
            if split == 'training':
                x_train.append(data)
                y_train.append(label)
            else:
                x_test.append(data)
                y_test.append(label)

    x_train, y_train = np.concatenate(x_train), np.concatenate(y_train)
    x_test, y_test = np.concatenate(x_test), np.concatenate(y_test)
    print('x_train.shape {} x_test.shape {}'.format(x_train.shape, x_test.shape))

    x_train = x_train / x_train.max(axis=(1, 2), keepdims=True)
    x_test = x_test / x_test.max(axis=(1, 2), keepdims=True)

    x_train = (x_train * 255.).astype(np.uint8)
    x_test = (x_test * 255.).astype(np.uint8)

    with h5py.File(cache_file, 'w') as f:
        f.create_dataset('x_train', data=x_train)
        f.create_dataset('y_train', data=y_train)
        f.create_dataset('x_test', data=x_test)
        f.create_dataset('y_test', data=y_test)
        print('saved to {}'.format(cache_file))

    return (x_train, y_train), (x_test, y_test)


# Data preprocessing function for PyTorch
def preprocess_data(x_train, x_test, img_size, use_subset=False):
    # Define transformations (resize, convert grayscale to 1 channel, normalize, and convert to tensor)
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((img_size, img_size)),  # Resize image to expected size
        transforms.Grayscale(num_output_channels=1),  # Keep the image in grayscale (1 channel)
        transforms.ToTensor(),  # Convert to Tensor
        transforms.Normalize(mean=[0.485], std=[0.229])  # Standard ResNet mean and std for grayscale
    ])

    # Apply the transformation to both train and test datasets
    x_train_transformed = [transform(x) for x in x_train]
    x_test_transformed = [transform(x) for x in x_test]

    # Convert to PyTorch tensors by stacking the list of tensors
    x_train_tensor = torch.stack(x_train_transformed)
    x_test_tensor = torch.stack(x_test_transformed)

    # Convert labels to tensors
    y_train_tensor = torch.tensor(y_train)
    y_test_tensor = torch.tensor(y_test)

    # If use_subset is True, use only 10% of the training data
    if use_subset:
        subset_size = int(0.1 * len(x_train_tensor))
        x_train_tensor = x_train_tensor[:subset_size]
        y_train_tensor = y_train_tensor[:subset_size]

    return x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor



# Profiling function to track time and memory usage
def profile_fn(fn):
    def wrapper(*args, **kwargs):
        # Start time and memory tracking
        start_time = time.time()
        process = psutil.Process(os.getpid())

        # Run the function
        result = fn(*args, **kwargs)

        # Calculate time taken
        end_time = time.time()
        time_taken = end_time - start_time

        # Memory usage
        memory_used = process.memory_info().rss / 1024 ** 2  # in MB
        return result, time_taken, memory_used
    return wrapper


# Model setup: ResNet-18
# Model setup: ResNet-18
def setup_resnet18(num_classes):
    # Load ResNet-18 architecture without pretrained weights
    model = models.resnet18(pretrained=False)

    # Modify the first convolutional layer to accept 1-channel input (grayscale)
    model.conv1 = nn.Conv2d(1, model.conv1.out_channels, kernel_size=3, stride=1, padding=1, bias=False)

    # Replace the fully connected layer to match the number of classes
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # Move the model to the appropriate device (GPU or CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    return model, device


# Define inference function
@profile_fn
def classify_image(model, image_tensor, device):
    model.eval()
    with torch.no_grad():
        output = model(image_tensor.to(device))
        prob = torch.nn.functional.softmax(output, dim=1)
        pred = torch.argmax(prob, dim=1).item()
        confidence = prob[0][pred].item()
    return pred, confidence


# Training function
def train_resnet18(model, train_loader, criterion, optimizer, device, num_epochs=2):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

        avg_loss = running_loss / len(train_loader)
        accuracy = 100 * correct_preds / total_preds
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')


# Main Execution
if __name__ == '__main__':
    # Setup dataset and configuration
    datadir = '/content/dataset/data'
    dataset = 'MNIST'
    num_classes = 10
    img_size = 28

    # Load and preprocess data (your existing data loading code)
    (x_train, y_train), (x_test, y_test) = load_data(dataset, num_classes, datadir)
    x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor = preprocess_data(x_train, x_test, img_size, use_subset=True)

    # Create DataLoader for train and test
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Setup the model
    model, device = setup_resnet18(num_classes)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Train the model
    train_resnet18(model, train_loader, criterion, optimizer, device, num_epochs=10)

    # Classify a single image and get time and memory usage
single_image = x_test_tensor[0].unsqueeze(0)  # Select a sample image from the test set (ensure it has a batch dimension)

# Get prediction, confidence, time taken, and memory usage
result, time_taken, mem_used = classify_image(model, single_image, device)
pred, conf = result  # Unpack prediction and confidence from result

print(f"Prediction: {pred}, Confidence: {conf:.4f}")
print(f"Time taken for classification: {time_taken:.4f} seconds")
print(f"Memory used: {mem_used:.2f} MB")



loaded from cache file data: x_train (60000, 28, 28) x_test (10000, 28, 28)
Epoch [1/10], Loss: 0.0676, Accuracy: 98.87%
Epoch [2/10], Loss: 0.0029, Accuracy: 99.98%
Epoch [3/10], Loss: 0.0011, Accuracy: 100.00%
Epoch [4/10], Loss: 0.0007, Accuracy: 100.00%
Epoch [5/10], Loss: 0.0005, Accuracy: 100.00%
Epoch [6/10], Loss: 0.0004, Accuracy: 100.00%
Epoch [7/10], Loss: 0.0003, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0002, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0002, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0002, Accuracy: 100.00%
Prediction: 0, Confidence: 0.9999
Time taken for classification: 0.0187 seconds
Memory used: 2064.97 MB
